In [1]:
import numpy as np
import os.path as op
import pandas as pd
import plotly
import seaborn as sns

from AFQ.api.group import GroupAFQ

/home/groups/jyeatman/software/ethanroy/envs/pyafq_eng/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pyafq_output_path = "/scratch/users/ethanroy/longitudinal_vanderbilt_colab/input_fmap_rev/derivatives/qsirecon"

In [2]:
## Helper Functions from:
## https://github.com/yeatmanlab/pyAFQ/blob/19fe6cd732601879f556d814218acafe365138f8/AFQ/api/group.py

def combine_list_of_profiles(profile_fnames):
    """
    Combine tract profiles from different subjects / sessions
    into one CSV.
    Parameters
    ----------
    profile_fnames : list of str
        List of csv filenames.
    Returns
    -------
    Ouput CSV's pandas dataframe.
    """
    dfs = []
    for fname in profile_fnames:
        profiles = pd.read_csv(fname)
        profiles['subjectID'] = fname.split('sub-')[1].split('/')[0][:-4]
        if 'ses' in fname:
            session_name = fname.split('ses')[1].split('/')[0]
        else:
            session_name = 'unknown'
        profiles['sessionID'] = session_name
        dfs.append(profiles)

    return clean_pandas_df(pd.concat(dfs))

# get rid of unnecessary columns in df
def clean_pandas_df(df):
    df = df.reset_index(drop=True)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [5]:
import glob

tract_profiles_csv = []

for f in glob.glob(op.join(pyafq_output_path,'**/*-AFQ_desc-profiles_dwi.csv'), recursive=True):
    tract_profiles_csv.append(f)
    

In [34]:
combined_profiles_df = combine_list_of_profiles(tract_profiles_csv)


In [35]:
combined_profiles_df = combined_profiles_df.rename(columns={'subjectID':'subject'})
combined_profiles_df['subject'] = 'sub-'+combined_profiles_df['subject']
combined_profiles_df['subjectID'] = combined_profiles_df['subject']+'ses'+combined_profiles_df['sessionID']



,tractID,nodeID,dti_fa,dti_md,dti_rd,dti_ad,subject,sessionID,subjectID
0,ATR_L,0,0.211710,0.000746,0.000663,0.000913,sub-LD4063,1,sub-LD4063ses1
1,ATR_L,1,0.242015,0.000731,0.000637,0.000921,sub-LD4063,1,sub-LD4063ses1
2,ATR_L,2,0.264670,0.000720,0.000617,0.000926,sub-LD4063,1,sub-LD4063ses1
3,ATR_L,3,0.281407,0.000711,0.000603,0.000926,sub-LD4063,1,sub-LD4063ses1
4,ATR_L,4,0.288651,0.000704,0.000596,0.000921,sub-LD4063,1,sub-LD4063ses1


In [36]:
combined_profiles_df.to_csv('/scratch/users/ethanroy/longitudinal_vanderbilt_colab/input_fmap_rev/derivatives/qsirecon/combined_tract_profiles_rev.csv')
